In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.models import Model, load_model, Sequential
from keras.applications.xception import Xception
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

DATA_DIR = 'dataset_50_class/'
TRAIN_DIR = 'train_50_class/'
TEST_DIR = 'test_50_class/'
IMG_SIZE = 128

Using TensorFlow backend.


In [2]:
def get_dict():
    MAPPING_DICT = {}
    i = 0

    for class_label in os.listdir(DATA_DIR):
        MAPPING_DICT[i] = class_label
        i += 1
    print(MAPPING_DICT)
    
    return MAPPING_DICT

In [3]:
def label_img(img):
    img_class = re.split("[_.]", img)[1]
    mapping_dict = get_dict()    
    i = 0
    
    for key,val in mapping_dict.items():
        if val == img_class:
            i = key
            break
    
    a = []
    for i in range(50):
        a.append(0)
    a[i] = 1
    
    return a

In [4]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data_vgg16.npy', training_data)
    return training_data

In [5]:
#train = create_train_data()
train = np.load("train_data_vgg16.npy", allow_pickle=True)

In [6]:
def create_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        label = label_img(img)
        path = os.path.join(TEST_DIR,img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),np.array(label)])
    shuffle(testing_data)
    np.save('test_data_vgg16.npy', testing_data)
    return testing_data

In [7]:
#test = create_test_data()
test = np.load("test_data_vgg16.npy", allow_pickle=True)

In [8]:
train_X_list = []
train_Y_list = []
test_X_list = []
test_Y_list = []

In [9]:
for i in range(len(train)):
    train_X_list.append(train[i][0].reshape(IMG_SIZE, IMG_SIZE, 3))
    train_Y_list.append(train[i][1])

for i in range(len(test)):
    test_X_list.append(test[i][0].reshape(IMG_SIZE, IMG_SIZE, 3))
    test_Y_list.append(test[i][1])

In [10]:
train_X = np.array(train_X_list)
train_Y = np.array(train_Y_list)
test_X = np.array(test_X_list)
test_Y = np.array(test_Y_list)

In [11]:
model = Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=train_X.shape[1:], pooling='max', classes=50)


83689472/83683744 [==============================] - 130s 2us/step


In [13]:
model.save("xception_2class.h5")
model = load_model("xception_2class.h5")
model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [14]:
for layer in model.layers[:-5]:
    layer.trainable = False
    
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7fceb0b735f8> False
<keras.layers.convolutional.Conv2D object at 0x7fceb0b73668> False
<keras.layers.normalization.BatchNormalization object at 0x7fceb0b736d8> False
<keras.layers.core.Activation object at 0x7fceb0b73908> False
<keras.layers.convolutional.Conv2D object at 0x7fceb0b73940> False
<keras.layers.normalization.BatchNormalization object at 0x7fceb0b73ac8> False
<keras.layers.core.Activation object at 0x7fceb0b73c18> False
<keras.layers.convolutional.SeparableConv2D object at 0x7fceb0b73d30> False
<keras.layers.normalization.BatchNormalization object at 0x7fceb0b73d68> False
<keras.layers.core.Activation object at 0x7fce9bc82198> False
<keras.layers.convolutional.SeparableConv2D object at 0x7fce9bc821d0> False
<keras.layers.normalization.BatchNormalization object at 0x7fce9bc82438> False
<keras.layers.convolutional.Conv2D object at 0x7fce9bc825c0> False
<keras.layers.pooling.MaxPooling2D object at 0x7fce9bc82748> False
<keras.la

In [16]:
model_final = Sequential()

model_final.add(model)
model_final.add(Dense(1024, activation='relu', name='fc1'))
model_final.add(Dropout(rate=0.4))
model_final.add(Dense(1024, activation='relu', name='fc2'))
model_final.add(Dropout(rate=0.4))
model_final.add(Dense(50, activation='softmax', name='fc3'))

In [17]:
adam = Adam(lr=0.0001)

model_final.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model_final.fit(train_X, train_Y, epochs=10, shuffle=True, validation_data=(test_X, test_Y), verbose=2)


Train on 19138 samples, validate on 8233 samples
Epoch 1/10


KeyboardInterrupt: 

In [19]:
predict = model.evaluate(test_X, test_Y, verbose=2)

print("loss = " + str(predict[0]))
print("accuracy = " + str(predict[1]))

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.